# Demoing GPT to BlogPost

In [1]:
!pip install --upgrade openai

   ---------------------------------------- 0.0/374.1 kB ? eta -:--:--
   --------------------------------- ------ 317.4/374.1 kB 6.5 MB/s eta 0:00:01
   ---------------------------------------- 374.1/374.1 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.43.0
    Uninstalling openai-1.43.0:
      Successfully uninstalled openai-1.43.0


In [2]:
from openai import OpenAI
import random
import os
from dotenv import load_dotenv
import json

load_dotenv()
gpt = os.getenv('gpt_token')
org = os.getenv('gpt_org')
client = OpenAI(api_key=gpt, organization=org)

def read_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return [json.loads(line) for line in file]

examples = read_jsonl("../example_docs/blog_posts.jsonl")

seo_terms = []
with open('../example_docs/seo_key_topics.txt', 'r') as file:
    seo_terms = [line.strip() for line in file]

seo_topics = random.choice(seo_terms)

sample = []
for i in range(0,9):
    index = random.randint(0, len(examples))
    sample.append(examples[index])
    
examples_str = '\n'.join(json.dumps(example, indent=4) for example in sample[0:8])

master_prompt = f"""
            You are a real-estate blog writer. Given examples of blog posts and also extra data accumulated from external
            sources, you will write a unique real estate blog post, making use of SEO keywords to make the blog post stand out. In the blog post itself,
            you will provide:
                - URL: a url for the blog post that follows SEO standards
                - Title: A title of what will covered in the blog post. The title should not appear in the content section.
                - Content: The content of the blog post. It should consist of at least 5000 words. Make sure that there is a lot to say 
                        within categories and subcategories, making it an information read for the reader. Also do not put the title here.
                - SEO Terms: A list of the SEO keywords used in the blog post
            
            Depending on what is asked of you, based on how many blogposts the client requests from, you will always default to one blog post.
            If the user asks for a specific amount, then provide that amount of blogposts in a listed array. However, in the output, you should respond
            with a json object of this structure:
                "url": "The blog post url",
                "title": "The blog post title",
                "content": "The blog post content",
                "seo_terms": ["keyword1", "keyword2"]
            Please follow the strucure and put a comma after every single key field. The content should be one whole string and any 
            kind of special character like \n for newline should be integrated within the content string to provide for that newline.
            You do not have to tell me or put into a comment that this is a json, the user will already know its a json.
            Please make sure that you follow the structure above and that something like seo terms is not found within "content", but should rather be 
            in the "seo_terms" key of the object body. I REPEAT DO NOT WRITE THE SEO TERMS WITHIN THE CONTENT IN MARKDOWN. IT SHOULD ONLY BE SEEN
            WITHIN "seo_terms". Please make sure that you are not using external links and link them inside the blog post you generate. 
            Please do not mention anything promotional or any footers regarding where the blog post is getting its data from, 
            how it's written, or anything not related to what a blog post is supposed to write. Please do not write a table of contents. 
            You are simply writing content. Please also make use of real estate SEO terms from example blog posts that may be given from the user.
            """

user_prompt = f"""
            Hello, I would like you to help me generate a unique real estate blog post. May you please write a blog with this topic: 
            {seo_topics}
            Here are examples of how real estate blog posts that I saw and would hope you would be able to come up with something unique that consists 
            of similar topics of the examples: {examples_str}. Follow the structure of the examples, and use some context from the examples, but use 
            the context and create your own blog post. And please provide a lot of numbers and statistics. Write the blog around the current month of 
            August 2024. Make the blog location specific. The blog should based in Austin, Texas. Make sure that SEO keywords are also seen frequently.
            Write the blog in HTML format as well and follow SEO formatting.
            """
# print(user_prompt)
tools = [
    {
        "type": "function",
        "function": {
            "name": "generate_blogpost",
            "description": "Generate a detailed real estate blog post based on user request and make use of SEO terms as keywords when constructing url, title and content",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "Includes url for real estate blogpost that engages SEO keywords of real estate based on the topic.",
                    },
                    "title": {
                        "type": "string",
                        "description": "Includes title for real estate blogpost that engages SEO keywords of real estate based on the topic. The title should not appear in the content section",
                    },
                    "content": {
                        "type": "string",
                        "description": "Includes content for real estate blogpost that engages SEO keywords of real estate based on the topic. Put this is HTML and make sure to follow SEO format. Do not put the title in the content. The title should only appear in the title section",
                    },
                    "seo_terms": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "Provide the list of SEO terms used when creating the real estate blogpost",
                    }
                },
                "required": [
                    "url",
                    "title",
                    "content",
                    "seo_terms",
                ]
            }
        }
    }
]
# print(user_prompt)
response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": master_prompt},
                    {"role": "user", "content": user_prompt},
                ],
                tools=tools,
                tool_choice="required",
                max_tokens=4096,
                temperature=0.7,
            )

# SEO Agent next - follow SEO structure
# Humanizer Agent
# Reflection of Agent
# Pushing into Wordpress

In [5]:
response_dump = response.model_dump()
choices = response_dump.get("choices",[])
if choices:
    message = choices[0].get("message", {})
    tool_calls = message.get("tool_calls", [])
    if tool_calls:
        function = tool_calls[0].get("function", {})
        arguments = function.get("arguments", {})
        if arguments:
            response_json = arguments
try:
    data = json.loads(response_json)
except:
    print("fail")

In [6]:
print("Topic of choice: ", seo_topics)
print(data["content"])

Topic of choice:  Top Reasons to Move to Fremont: The Emerging Silicon Valley Suburb
<h2>Introduction</h2>
<p>As the real estate market continues to evolve, Fremont, California, has emerged as one of the most desirable suburbs in the Silicon Valley area. Known for its exceptional standard of living, educational opportunities, and economic growth, Fremont is attracting a diverse range of residents from across the country. In this blog post, we will explore the top reasons why moving to Fremont could be a life-changing decision for you and your family.</p>

<h2>1. Thriving Job Market</h2>
<p>Fremont's proximity to Silicon Valley makes it a hotbed for job opportunities, especially in the tech industry. Major companies like Tesla, Lam Research, and Western Digital have their headquarters or significant offices in the area, offering numerous job opportunities. The city's unemployment rate is notably low at just 2.9%, compared to the national average of 3.7% as of August 2024. This robust jo

In [7]:
master_prompt_2 = """
        You are a real estate blog post reviewer. Your job is to read through the contents of real estate blog posts and provide suggestions
        on how to make it better in terms of content, SEO, and providing more stats or information that will make the real estate blogpost more 
        accurate to current data.
"""

user_prompt_2 = f"""
        I wrote a real estate blog post based on this topic: {seo_topics}. Here is the blog post that I wrote: {data["content"]}.
        Can you give me suggestions to make it better in terms of content, writing style, and better SEO suggestions.
"""

tool2 = [
    {
        "type": "function",
        "function": {
            "name": "generate_edits",
            "description": "Generate a detailed list of suggestions to make the real estate blog post provided by the user better when it comes to content, data, and SEO rules",
            "parameters": {
                "type": "object",
                "properties": {
                    "suggestions": {
                        "type": "string",
                        "description": "List of suggestions for improvements on the real estate blog post"
                    }
                },
                "required": [
                    "suggestions",
                ]
            }
        }
    }
]

response2 = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": master_prompt_2},
                    {"role": "user", "content": user_prompt_2},
                ],
                tools=tool2,
                tool_choice="required",
                max_tokens=4096,
                temperature=0.7,
            )

In [8]:
response_dump2 = response2.model_dump()
choices2 = response_dump2.get("choices",[])
if choices2:
    message2 = choices2[0].get("message", {})
    tool_calls2 = message2.get("tool_calls", [])
    if tool_calls2:
        function2 = tool_calls2[0].get("function", {})
        arguments2 = function2.get("arguments", {})
        if arguments2:
            response_json2 = arguments2
try:
    data2 = json.loads(response_json2)
    print(data2['suggestions'])
except:
    print("fail")

1. **Content Enhancement:**
   - Provide more recent and specific data points for each section.
   - Include quotes or testimonials from residents or real estate experts.
   - Add a section on entertainment and nightlife options.
   - Discuss the healthcare facilities and services available in Fremont.
   - Include information about the climate and weather patterns.

2. **SEO Improvement:**
   - Incorporate relevant keywords such as "Fremont real estate," "Silicon Valley suburb," "moving to Fremont," and "living in Fremont" throughout the text.
   - Optimize meta descriptions with primary keywords.
   - Use header tags (H1, H2, H3) effectively to organize content and improve readability.
   - Add internal and external links to relevant resources and authoritative sites.
   - Use alt text for any images included in the blog post.

3. **Writing Style:**
   - Use shorter paragraphs and bullet points to improve readability.
   - Add subheadings within each section for better structure.
   

In [9]:
master_prompt_3 = """
    You are a skilled real estate blog post writer tasked with crafting the final draft of a blog post. You will receive the first draft of the 
    blog post along with a set of suggestions provided by a real estate blog post reviewer. Your task is to carefully review the first draft 
    and fully incorporate the suggestions provided to create a polished final draft. Enhance the clarity, engagement, and natural flow of 
    the content, ensuring it reads smoothly and appeals to the target audience. Remember to humanize the content, making it feel conversational and 
    approachable while retaining the key messages and SEO elements. There should be a noticeable difference from the original draft.
"""


user_prompt_3 = f"""
I wrote an initial draft of a real estate blog post on the topic: {seo_topics}. I want to improve this draft based on suggestions I received from a real estate blog post reviewer.

Here is the first draft of the blog post:
{data['content']}

Here are the suggestions:
{data2['suggestions']}

Please incorporate these suggestions into the final draft, enhancing readability, tone, and structure. Ensure the final draft is more engaging, flows naturally, and is well-aligned with the topic and SEO objectives.
"""


tools_3 = [
    {
        "type": "function",
        "function": {
            "name": "generate_reviewed_blogpost",
            "description": "Generate a final draft of a real estate blogpost that was provided by the user and rewrite it with the suggestions they provided",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "Includes url for real estate blogpost that engages SEO keywords of real estate based on the topic. Only provide everything after the domain name",
                    },
                    "title": {
                        "type": "string",
                        "description": "Includes title for real estate blogpost that engages SEO keywords of real estate based on the topic.",
                    },
                    "content": {
                        "type": "string",
                        "description": "Includes content for real estate blogpost that engages SEO keywords of real estate based on the topic. Can you use html for this. Only provide the content of blog post do not add your own thoughts outside of the blog posts",
                    },
                    "seo_terms": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "Provide the list of SEO terms used when creating the real estate blogpost",
                    }
                },
                "required": [
                    "url",
                    "title",
                    "content",
                    "seo_terms",
                ]
            }
        }
    }
]

response3 = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": master_prompt_3},
                    {"role": "user", "content": user_prompt_3},
                ],
                tools=tools_3,
                tool_choice="required",
                max_tokens=4096,
                temperature=1.0,
            )

In [10]:
response_dump3 = response3.model_dump()
choices3 = response_dump3.get("choices",[])
if choices3:
    message3 = choices3[0].get("message", {})
    tool_calls3 = message3.get("tool_calls", [])
    if tool_calls3:
        function3 = tool_calls3[0].get("function", {})
        arguments3 = function3.get("arguments", {})
        if arguments3:
            response_json3 = arguments3
try:
    print(response_json3)
    data3 = json.loads(response_json3)
except:
    print("fail")

{"url": "/top-reasons-move-fremont-emerging-silicon-valley-suburb", "title": "Top Reasons to Move to Fremont: The Emerging Silicon Valley Suburb", "content": "<h2>Introduction</h2>\n<p>As the real estate market continues to evolve, Fremont, California, has emerged as one of the most desirable suburbs in the Silicon Valley area. Known for its exceptional standard of living, educational opportunities, and economic growth, Fremont is attracting a diverse range of residents from across the country. If you're considering moving to Fremont, this blog post will provide valuable insights into why this city could become your new home.</p>\n\n<h2>1. Thriving Job Market</h2>\n<p>Thanks to its close proximity to Silicon Valley, Fremont is a hotspot for job opportunities, particularly in the technology sector. Major companies like Tesla, Lam Research, and Western Digital have headquarters or significant operations in Fremont, creating a wealth of employment options. The city's low unemployment rate

In [39]:
master_prompt_4 = """
As a humanizing GPT agent, your task is to rewrite blog posts in a conversational, relatable tone. Here's how:

### 1. Be Conversational
    - Write like you're explaining this to a friend, with simple, friendly language.
    - Add questions and casual phrases to make it feel approachable.
    - Use contractions (like "don't" or "you’re") to sound natural.

### 2. Show Emotion
    - Acknowledge feelings like stress, excitement, or frustration, depending on the topic.
    - Use phrases that show empathy or understanding, like "I know how tough that can be" or "We've all been there."

### 3. Personal Touch
    - Include first-person pronouns and hypothetical stories or examples.
    - Make it sound like you’re speaking from experience (e.g., “I’ve seen this a lot in my own work”).
    - Provide 

### 4. Light Humor
    - Keep the tone light by adding small jokes or humorous remarks where appropriate.
    - Avoid being overly formal or dry—keep it engaging and fun.
"""



user_prompt_4 = f"""
I’ve written a real estate blog post on {seo_topics}, but it sounds too formal and robotic. Could you please make it more conversational, 
relatable, and personal? I want it to still sound professional, but also like I'm talking to a colleague. Add contractions, personal anecdotes, and a light touch of humor where 
appropriate. Here’s the content I want humanized: {data3["content"]}.
"""



tools_4 = [
    {
        "type": "function",
        "function": {
            "name": "generate_humanized_blogpost",
            "description": "Generate a final draft of a real estate blogpost that was provided by the user and rewrite making it feel humanized, relatable, and personal.",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "Includes url for real estate blogpost that engages SEO keywords of real estate based on the topic. Only provide everything after the domain name",
                    },
                    "title": {
                        "type": "string",
                        "description": "Includes title for real estate blogpost that engages SEO keywords of real estate based on the topic.",
                    },
                    "content": {
                        "type": "string",
                        "description": "Includes humanized content for real estate blogpost that engages SEO keywords of real estate based on the topic. Can you use html for this.",
                    },
                    "seo_terms": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "Provide the list of SEO terms used when creating the real estate blogpost",
                    }
                },
                "required": [
                    "url",
                    "title",
                    "content",
                    "seo_terms",
                ]
            }
        }
    }
]

response4 = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": master_prompt_4},
                    {"role": "user", "content": user_prompt_4},
                ],
                tools=tools_4,
                tool_choice="required",
                max_tokens=4096,
                temperature=0.8,
            )

In [40]:
response_dump4 = response4.model_dump()
choices4 = response_dump4.get("choices",[])
if choices4:
    message4 = choices4[0].get("message", {})
    tool_calls4 = message4.get("tool_calls", [])
    if tool_calls4:
        function4 = tool_calls4[0].get("function", {})
        arguments4 = function4.get("arguments", {})
        if arguments4:
            response_json4 = arguments4
try:
    print(response_json4)
    data4 = json.loads(response_json4)
except:
    print("fail")

{"url":"/top-reasons-to-move-to-fremont","title":"Top Reasons to Move to Fremont: The Emerging Silicon Valley Suburb","content":"<h2>Introduction</h2>\n<p>Hey there! Thinking about making a move? Well, let me tell you about Fremont, California. It's quickly becoming the place to be in the Silicon Valley area. Imagine a spot with a fantastic standard of living, top-notch schools, and a booming economy. Sounds great, right? If Fremont's on your radar, you’re in luck—this blog post is going to give you the lowdown on why this city is such a gem.</p>\n\n<h2>1. Thriving Job Market</h2>\n<p>Alright, let's talk jobs. Being so close to Silicon Valley, Fremont is a goldmine for job opportunities, especially in tech. Companies like Tesla, Lam Research, and Western Digital have major operations here. That means tons of job options! Plus, with an impressively low unemployment rate of 2.4% (compared to the national average of 3.5%), you're looking at a pretty strong job market. Jeanine Smith, a loc

In [41]:
print(data4["content"])

<h2>Introduction</h2>
<p>Hey there! Thinking about making a move? Well, let me tell you about Fremont, California. It's quickly becoming the place to be in the Silicon Valley area. Imagine a spot with a fantastic standard of living, top-notch schools, and a booming economy. Sounds great, right? If Fremont's on your radar, you’re in luck—this blog post is going to give you the lowdown on why this city is such a gem.</p>

<h2>1. Thriving Job Market</h2>
<p>Alright, let's talk jobs. Being so close to Silicon Valley, Fremont is a goldmine for job opportunities, especially in tech. Companies like Tesla, Lam Research, and Western Digital have major operations here. That means tons of job options! Plus, with an impressively low unemployment rate of 2.4% (compared to the national average of 3.5%), you're looking at a pretty strong job market. Jeanine Smith, a local employment consultant, says, "Fremont's job market is dynamic and growing, offering a range of opportunities for professionals at 